In [1]:
import pandas as pd
import os
import fnmatch
import sqlite3

os.listdir('./output')
pd.set_option("display.max_rows",1000)
pd.set_option("display.max_columns", 100)

terms = pd.read_csv('./input/industry_terms.csv')
terms_dict = pd.melt(terms).dropna().reset_index(drop=True).set_index('value').to_dict()['variable']

#donors = pd.read_csv('./output/donors.tsv', sep='\t')
contribs = pd.read_csv('./output/contributors2.tsv', sep='\t')


def clean_input_df(input_df, col_name):
    input_df = input_df.groupby('bonica_cid').head(1).dropna().reset_index(drop=True)
    input_df.loc[:,col_name] = input_df.loc[:,col_name].str.lower()
    input_df.iloc[:,0] = input_df.iloc[:,0].astype(int)
    # Remove Names and Individual Donors
    no_commas = input_df[~input_df.loc[:,col_name].str.contains(",")]
    inc_commas = input_df[(input_df.loc[:,col_name].str.contains(",")) & (input_df.loc[:,col_name].str.contains("inc")) & (input_df.loc[:,col_name].str.contains("llc"))]
    long_commas = input_df[(input_df.loc[:,col_name].str.contains(",")) & (input_df.loc[:,col_name].str.len()>25) & 
                           ~(input_df.loc[:,col_name].str.contains("cash")) & ~(input_df.loc[:,col_name].str.contains("unreadable"))]
    return pd.concat([no_commas, inc_commas, long_commas])

df ='contribs'
col ='contributor_name'
def get_industry_df(df_str, col):
    industry_df = pd.DataFrame()
    for name, column in terms.iteritems():
        search_str = ''
        for term in column.dropna():
            if len(search_str) == 0:
                search_str = ("{df}.{column}.str.contains('{term}')".format(df=df_str, column=col, term=term))
            else:
                search_str += ("| {df}.{column}.str.contains('{term}')".format(df=df_str, column=col, term=term))
        term_df = eval(f"{df_str}[{search_str}]" + '\n')
        term_df.loc[:,'industry'] = name
        industry_df = pd.concat([industry_df, term_df])
        print(name)
    return industry_df

In [2]:
contribs_industry = get_industry_df('contribs', 'contributor_name')

donors = pd.read_csv('./output/donors.tsv', sep='\t')
donors = clean_input_df(donors, 'most_recent_contributor_name')

donors_industry = get_industry_df('donors', 'most_recent_contributor_name')
donors_industry.rename(columns={'most_recent_contributor_name':'contributor_name'})

industry = pd.concat([contribs_industry, donors_industry.rename(columns={'most_recent_contributor_name':'contributor_name'})])
industry = industry.drop_duplicates()

/home/yune/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/yune/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


finance
arms
pharma_med
education
agriculture
tobacco


KeyboardInterrupt: 

In [123]:
industry.loc[:,'industry'].value_counts()

org_labor      172619
finance        162931
pharma_med     148766
manufactur     117028
energy         102970
political       66066
agriculture     46838
education       20361
entertain       13154
native          12376
telecoms        10230
arms            10005
womens           5821
environment      5316
israel           2758
tobacco          2265
technology       1318
lgbt              301
Name: industry, dtype: int64

In [125]:
industry.set_index('bonica_cid', inplace=True)

,contributor_name,industry
bonica_cid,,
1.156000e+03,mich. savings & loan league public affairs com...,finance
1.875000e+03,u s league savings association political elect...,finance
2.253000e+03,thriftpac national association of mutual savin...,finance
2.642000e+03,morgan murphy for congress committee,finance
3.830000e+03,chemical bank fund for good government,finance
4.010000e+03,savings associations' league political action ...,finance
4.515000e+03,h. & r.e. & b.i.u. t.i.p. to insure progress a...,finance
4.861000e+03,unite here tip to insure progress campaign com...,finance
4.861000e+03,unitehere t i p to insure progress,finance


In [126]:
conn = sqlite3.connect('db/dime.sqlite3')
c = conn.cursor()


c.execute('DROP TABLE IF EXISTS industry')
c.execute('CREATE TABLE industry (bonica_cid  INTEGER PRIMARY KEY, contributor_name TEXT, industry TEXT);')
#c.execute('INSERT INTO industry(contributor_name, industry) VALUES (?, ?)', (str(industry.loc[:,'contributor_name']), str(industry.loc[:,'industry'])))


In [128]:
industry.groupby('bonica_cid').first().to_sql('industry', conn, if_exists='replace', index_label='id')

In [130]:
recipients = pd.read_sql('''SELECT * FROM industry I LIMIT 10''', conn)

,id,contributor_name,industry
0,18.0,ironworkers local union no. 25 political educa...,org_labor
1,42.0,illinois tool workers inc for better governmen...,org_labor
2,141.0,burton for congress campaign committee,political
3,422.0,american medial association pac ampac,pharma_med
4,489.0,"d r i v e political fund, teamsters local unio...",org_labor
5,505.0,glenn anderson campaign committee,political
6,547.0,kanas medical society pac,pharma_med
7,638.0,indiana medical political action committee,pharma_med
8,828.0,friends of ed jones,political
9,885.0,international union of painters and allied tra...,org_labor


In [143]:
recipients = pd.read_sql('''SELECT C.cycle, I.industry, C.bonica_rid, C.recipient_name, C.recipient_party, C.recipient_type,  
            C.recipient_state, C.seat, C.candidate_cfscore, C.bonica_cid, C.contributor_name, C.transaction_id, 
            C.transaction_type, C.efec_memo2, C.amount
           FROM industry I 
           INNER JOIN contribDB C on I.id = C.bonica_cid 
           WHERE amount > 200
           GROUP by C.cycle, I.industry, C.bonica_rid, C.recipient_name, C.recipient_party, C.recipient_type,  
            C.recipient_state, C.seat, C.candidate_cfscore, C.bonica_cid, C.contributor_name, C.transaction_id, 
            C.transaction_type, C.efec_memo2, C.amount''', conn)

In [144]:
# Earmarked funds like actblue only display totals in the total earmarked section.
def better_totals(input_df):
    input_df.loc[:,'efec_memo2'] = input_df.loc[:,'efec_memo2'].str.lower()
    total_earmarked = input_df[input_df.loc[:,'efec_memo2'].str.contains("total earmarked", na=False)].groupby('contributor_name').first().reset_index()
    remaining = input_df[~input_df.contributor_name.isin(total_earmarked.loc[:,'contributor_name'])]
    return pd.concat([total_earmarked, remaining])

#total_earmarked.amount.sum() + remaining.amount.sum()

In [142]:
recipients

,cycle,industry,bonica_rid,recipient_name,recipient_party,recipient_type,recipient_state,seat,candidate_cfscore,transaction_id,transaction_type,efec_memo2,amount
0,1980,agriculture,,1979 republican senate house dinner,200,comm,dc,federal:committee,NaN,indv:1980:186813,15,None,1000.00
1,1980,agriculture,,1979 republican senate house dinner,200,comm,dc,federal:committee,NaN,indv:1980:186855,15,None,250.00
2,1980,agriculture,,1980 united republican committee,200,comm,dc,federal:committee,NaN,indv:1980:323969,15,None,1000.00
3,1980,agriculture,,countdown to victory,200,comm,dc,federal:committee,NaN,indv:1980:273754,15,None,500.00
4,1980,agriculture,,national congressional club,,comm,nc,federal:committee,NaN,indv:1980:303065,15,None,1000.00
5,1980,agriculture,,national congressional club,,comm,nc,federal:committee,NaN,indv:1980:303442,15,None,2000.00
6,1980,agriculture,,national congressional club,,comm,nc,federal:committee,NaN,indv:1980:303497,15,None,2000.00
7,1980,agriculture,,national congressional club,,comm,nc,federal:committee,NaN,indv:1980:303498,15,None,1000.00
8,1980,agriculture,,national congressional club,,comm,nc,federal:committee,NaN,indv:1980:303499,15,None,500.00
9,1980,agriculture,,ohio republican finance committee,200,comm,oh,federal:committee,NaN,indv:1980:11113,15,None,1000.00


In [154]:
house_recipients = better_totals(recipients[recipients.loc[:,'seat'] == 'federal:house'])

house_industry_recipients = house_recipients.groupby(['bonica_rid','recipient_name', 'recipient_party', 'recipient_state','cycle','industry','candidate_cfscore']).sum().iloc[:,:-1]

In [156]:
house_industry_recipients.sort_values('amount', ascending=False)

,,,,,,,amount
bonica_rid,recipient_name,recipient_party,recipient_state,cycle,industry,candidate_cfscore,
cand1225,"boehner, john andrew",200,oh,2012,political,0.94,10134838.00
cand1349,"chambliss, c saxby",200,ga,2008,political,1.05,8775253.00
cand835,"kirk, mark steven",200,il,2010,political,0.59,6280963.00
cand36131,"kahn, roger f",100,ga,2004,political,-0.21,6032499.00
cand880,"harris, katherine",200,fl,2006,political,1.01,3952619.00
cand42611,"delaney, john k",100,md,2012,political,-0.69,3801734.10
cand1327,"brown, sherrod",100,oh,2006,political,-1.06,3713649.00
cand864,"cantor, eric ivan",200,va,2012,political,0.58,3582931.05
cand303,"paul, ron",200,tx,2008,political,1.58,3530004.00


In [160]:
house_industry_recipients.reset_index()[house_industry_recipients.reset_index().loc[:,'industry'] != 'political'].sort_values('amount', ascending=False)

,bonica_rid,recipient_name,recipient_party,recipient_state,cycle,industry,candidate_cfscore,amount
12403,cand1225,"boehner, john andrew",200,oh,2012,finance,0.94,3219622.34
12407,cand1225,"boehner, john andrew",200,oh,2012,org_labor,0.94,2788377.00
5335,cand1110,"west, allen b mr.",200,fl,2012,agriculture,1.29,2437080.00
37777,cand1509,"miller, gary g",200,ca,2012,finance,0.55,2414200.24
12408,cand1225,"boehner, john andrew",200,oh,2012,pharma_med,0.94,1913617.00
118666,cand688,"kanjorski, paul e",100,pa,2008,finance,-0.38,1901406.00
30539,cand1433,"sherman, brad",100,ca,2012,finance,-0.66,1894480.41
12418,cand1225,"boehner, john andrew",200,oh,2014,finance,0.94,1846264.18
7041,cand1153,"renacci, james b",200,oh,2012,org_labor,1.02,1754882.00
7026,cand1153,"renacci, james b",200,oh,2010,org_labor,1.02,1707918.00


In [113]:
# grayson = pd.read_sql('''SELECT *
#            FROM industry I 
#            INNER JOIN contribDB C on I.id = C.bonica_cid 
#            WHERE bonica_rid = 'cand1035' AND cycle = 2012
#             ORDER by amount DESC''', conn)
# grayson = grayson.iloc[:,2:]
# grayson2 = better_totals(grayson)

# grayson[grayson.loc[:, 'industry'] != 'political'].groupby('industry').amount.sum()

# grayson[grayson.loc[:, 'industry'] == 'education']

In [72]:
recipients.to_csv('output/recipient_industry.tsv', sep='\t')

In [71]:
recipients[recipients.loc[:,'seat'] == 'federal:house'].to_csv('output/house_recipient_industry.tsv', sep='\t')

In [25]:
c.execute("SELECT * FROM sqlite_master WHERE type='table';").fetchall()

[('table',
  'contribDB',
  'contribDB',
  2,
  'CREATE TABLE contribDB \n( "cycle" INTEGER,\n\t"transaction_id" TEXT,\n\t"transaction_type" TEXT,\n\t"amount" INTEGER,\n\t"date" TEXT,\n\t"bonica_cid" REAL,\n\t"contributor_name" TEXT,\n\t"contributor_lname" TEXT,\n\t"contributor_fname" TEXT,\n\t"contributor_mname" TEXT,\n\t"contributor_suffix" TEXT,\n\t"contributor_title" TEXT,\n\t"contributor_ffname" TEXT,\n\t"contributor_type" TEXT,\n\t"contributor_gender" TEXT,\n\t"contributor_address" TEXT,\n\t"contributor_city" TEXT,\n\t"contributor_state" TEXT,\n\t"contributor_zipcode" INTEGER,\n\t"contributor_occupation" TEXT,\n\t"contributor_employer" TEXT,\n\t"is_corp" TEXT,\n\t"recipient_name" TEXT,\n\t"bonica_rid" TEXT,\n\t"recipient_party" TEXT,\n\t"recipient_type" TEXT,\n\t"recipient_state" TEXT,\n\t"seat" TEXT,\n\t"election_type" TEXT,\n\t"latitude" REAL,\n\t"longitude" REAL,\n\t"gis_confidence" REAL,\n\t"contributor_district_90s" TEXT,\n\t"contributor_district_00s" TEXT,\n\t"contributor_d

In [9]:
donors.loc[:,'most_recent_contributor_name'] = donors.loc[:,'most_recent_contributor_name'].str.lower()

In [10]:
donors[~(donors.bonica_cid.isin(donors2.bonica_cid))].to_csv('output/individual_donors.tsv',sep='\t')

donors[(donors.most_recent_contributor_name.str.len()<20) & (donors.most_recent_contributor_name.str.contains(","))].to_csv('output/individual_donors.tsv',sep='\t')

NameError: name 'donors2' is not defined

In [16]:
# Old Way - extremely inefficient algo
# for index, row in contribs.iterrows():
#     for key,val in terms_dict.items():
#         try:
#             if fnmatch.fnmatch(row['contributor_name'], '*{}*'.format(key)):
#                 contribs.loc[index,'industry'] = val
#         except TypeError:
#             print('TypeError')
#     if index%5000==0:
#         print(index)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000
470000
475000
480000
485000
490000
495000
500000
505000
510000
515000
520000
525000
530000
535000
540000
545000
550000
555000
560000
565000
570000
575000
580000
585000
590000
595000
600000
605000
610000
615000
620000
625000
630000
635000
640000
645000
650000
655000
660000
665000
670000
675000
680000
685000
690000
695000
700000
705000
710000
715000
720000
725000
730

KeyboardInterrupt: 

In [21]:
contribs.loc[:3530000, :].dropna().to_csv('output/contrib3530000.tsv',sep='\t')

In [30]:
contribs[contribs.contributor_name.str.len()>20 & contribs.contributor_name.str.contains(",")]

,bonica_cid,contributor_name,industry


In [35]:
contribs[(contribs.contributor_name.str.len()<20) & (contribs.contributor_name.str.contains(","))].to_csv('output/individual_contribs.tsv',sep='\t')